Početak 2020-02-18

In [ ]:
import pandas as pd
from pandas import DataFrame as df
from google.colab import files

In [ ]:
start_date="2020-03-15"
end_date="2020-03-19"
wunderground_station_id="IPTUJ11"
station_id="DUDU1234"

In [ ]:
range_of_dates=pd.date_range(start=start_date, end=end_date)

In [ ]:
range_of_dates=list(map(lambda x: str(x)[:10],range_of_dates))

In [ ]:
print(range_of_dates)

['2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19']


In [ ]:
for date in range_of_dates:

  url=f"https://www.wunderground.com/dashboard/pws/{wunderground_station_id}/table/{date}/{date}/daily"
  dfs = pd.read_html(url,match="Precip. Rate.")
  data = pd.concat(dfs)
  data.dropna(inplace = True)
  data.to_csv(f"{wunderground_station_id}_{date}")

In [ ]:
files.download()

TypeError: ignored